<a href="https://colab.research.google.com/github/AirPro/Mod4HandsOnFreidrd/blob/main/Mod4HandsOnFreidrd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4 Hands On Traffic Sign Assignment
By: Robert Freid for the Deep Learning Mod 4 Assignment
## Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Mounting Google Drive Notes
Drive already mounted at /content/drive; <br>
To attempt to forcibly remount call: <br>
drive.mount('/content/drive',force_remout=True)
## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # to plot accuracy
import cv2
import tensorflow as tf
from PIL import Image # to read image data
import os
from sklearn.model_selection import train_test_split # to split training and testing data
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout 

## Read the images and save them and their corresponding labels in lists

In [ ]:
data = []
labels = []
classes = 43

# Copied Path: /content/drive/MyDrive/TrafficSignsData/myData

cur_path = "/content/drive/MyDrive/TrafficSignsData/myData"

# Retreiving the images and their labels

for i in range(classes):
  path = os.path.join(cur_path,str(i))
  images = os.listdir(path)
  for a in images:
    try:
      image = Image.open(path + '/' + a)
      image = image.resize((32, 32))
      image = np.array(image)
      data.append(image)
      labels.append(i)
    except:
      print('Error Loading Image')

## Save the data and labels as numpy arrays
## Divide training and testing sets

In [ ]:
data = np.array(data)
labels = np.array(labels)
print('Data.shape')
print(data.shape)
print('Labels.shape')
print(labels.shape)

# Splitting training and testing data

X_train, X_test, y_train, y_test= train_test_split(data, labels, test_size=0.2, random_state=42)
print('train_test_split results')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## One hot encodng for labels

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Define the Deep Learning Model 

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

## Show the Model Summary

In [ ]:
model.summary()

## Plot the Model Diagram

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', dpi=50,show_shapes=True, show_layer_names=True)

## Compile the Model

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Set Batch size and number of Epochs

In [ ]:
epochsNum = 10
batchSize = 100

## Train the model

In [ ]:
history=model.fit(X_train, y_train, epochs=epochsNum, batch_size=batchSize, validation_split=0.1, shuffle=True)

## Train versus Validation Training Loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'validation'], loc='upper right', fontsize='large')
plt.ylabel('loss', fontsize=16)
plt.xlabel('epoch', fontsize=16)
plt.yticks()
plt.xticks(np.arange(0, epochsNum))
plt.show()

## Train versus Validation Accuracy

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch', fontsize=16)
plt.ylabel('accuracy', fontsize=16)
plt.yticks(np.arange(0.0, 1.05, 0.1))#, step=0.5))
plt.xticks(np.arange(0, epochsNum))# +1 )) step=10))
plt.legend(['train', 'validation'], loc='lower right', fontsize='large')
plt.show()

## Test the Model

In [ ]:
pred_test = model.predict(X_test)
y_pred = pred_test.round()
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

## Decode the One Hot Encoder back to Labels

In [ ]:
decoded_y_test = tf.argmax(y_test, axis=1)
decoded_prediction = tf.argmax(pred_test, axis=1)

print(decoded_y_test.shape)
print(decoded_prediction.shape)

## Confusion Matrix

In [ ]:
tf.math.confusion_matrix(decoded_y_test, decoded_prediction)

## Confusion Matrix using Seaborn

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn

cm = confusion_matrix(decoded_y_test, decoded_prediction)

sn.heatmap(cm,annot=True) # fmt='d', xticklabels=labels[name],yticklabels=labels['name], cbar=False
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title("Confusion Matrix", fontsize=10)